In [ ]:
from os import urandom, getcwd
import hashlib, hmac, binascii
import ecc.ecc as ecc
import ecc.util as util

## Génération de la seed
Nous allons commencer en générant un peu d'entropie (une séquence de bits aléatoires)

In [ ]:
entropy = urandom(16)
print(entropy)

On pourrait aussi générer cette entropie avec une fonction de hashage pour la rendre déterministe

In [ ]:
_entropy = hmac.new(b"password", b"salt", digestmod=hashlib.sha512).digest()
print(_entropy)

Cette entropie originale peut être encodée dans une série de mots (mais c'est facultatif)

In [ ]:
class Mnemonic:
    def __init__(self, language):
        self.radix = 2048
        with open("%s/%s.txt" % (getcwd(), language), "r", encoding="utf-8") as f:
            self.wordlist = [w.strip() for w in f.readlines()]
        if len(self.wordlist) != self.radix:
            raise ConfigurationError(
                "Wordlist should contain %d words, but it contains %d words."
                % (self.radix, len(self.wordlist))
            )

    def to_mnemonic(self, data):
        if len(data) not in [16, 20, 24, 28, 32]:
            raise ValueError(
                "Data length should be one of the following: [16, 20, 24, 28, 32], but it is not (%d)."
                % len(data)
            )
        h = hashlib.sha256(data).hexdigest()
        b = (
            bin(int(binascii.hexlify(data), 16))[2:].zfill(len(data) * 8)
            + bin(int(h, 16))[2:].zfill(256)[:len(data) * 8 // 32]
        )
        '''
        print(f"binascii.hexlify(data): {binascii.hexlify(data)}")
        print(f"int(binascii.hexlify(data), 16): {int(binascii.hexlify(data), 16)}")
        print(f"bin(int(binascii.hexlify(data), 16)): {bin(int(binascii.hexlify(data), 16))}")
        print(f"bin(int(binascii.hexlify(data), 16))[2:]: {bin(int(binascii.hexlify(data), 16))[2:]}")
        print(f"bin(int(binascii.hexlify(data), 16))[2:].zfill(len(data) * 8): \
            {bin(int(binascii.hexlify(data), 16))[2:].zfill(len(data) * 8)}")
        print(f"bin(int(h, 16))[2:].zfill(256): {bin(int(h, 16))[2:].zfill(256)}")
        print(f"bin(int(h, 16))[2:].zfill(256)[:len(data) * 8 // 32]: \
            {bin(int(h, 16))[2:].zfill(256)[:len(data) * 8 // 32]}")

        print(f"h is {h}")
        print(f"b is {b}")
        '''
        result = []
        for i in range(len(b) // 11):
            # l'index est calculé en prenant 11 bits sur b et en le convertissant en int
            idx = int(b[i * 11 : (i + 1) * 11], 2)
            # on ajoute le mot de la liste se trouvant à l'indice
            result.append(self.wordlist[idx])
        result_phrase = " ".join(result)
        return result_phrase

In [ ]:
mnemonic = Mnemonic('english')
m = mnemonic.to_mnemonic(entropy)
print(m)

## Key stretching
Notre secret fait pour l'instant entre 128 et 256 bits, il faut l'étendre à 512 bits. 
On utilise une fonction "Password-Based Key Derivation Function 2" (PBKDF2), qui comme son nom l'indique prend un password et un "salt" pour générer un hash dont on peut choisir la longueur.
![](pics/hd1.jpg)

In [ ]:
def to_seed(mnemonic, passphrase=""):
    passphrase = "mnemonic" + passphrase
    mnemonic = mnemonic.encode("utf-8")
    passphrase = passphrase.encode("utf-8")
    stretched = hashlib.pbkdf2_hmac("sha512", mnemonic, passphrase, 1000)
    return stretched

In [ ]:
seed = to_seed(m)
print(seed)

## HD private key
On va (encore) hasher la seed pour obtenir notre clé privée étendue, celle qui permettra ensuite de dériver des milliards d'autres clés.
Cette clé "racine" comprend une clé privée "classique" (32 bytes) et un chaincode qui permet la dérivation à proprement parler.
![](pics/hd2.jpg)

In [ ]:
class HD_key:
    def __init__(self, seed):
        if len(seed) != 64:
            raise ValueError("Provided seed should have length of 64")
        ext_hd = hmac.new(b"Bitcoin seed", seed, digestmod=hashlib.sha512).digest()
        self.keys = ecc.PrivateKey(int.from_bytes(ext_hd[:32], 'big'))
        self.chaincode = ext_hd[32:]

    def __repr__(self):
        return f"The private key is {self.keys.wif()}\n" + \
            f"The public key is {binascii.hexlify(self.keys.point.sec())}\n" + \
            f"The chaincode is {binascii.hexlify(self.chaincode)}"


In [ ]:
hd_root = HD_key(seed)
print(hd_root)

## Encodage des clés étendues
Une clé privée ou publique est étendue lorsqu'elle comprend le chaincode. 
Il existe un standard d'encodage pour la représentation de ces clés.

In [ ]:
ENCODING_PREFIX = {
    "main": {
        "private": 0x0488ADE4,
        "public": 0x0488B21E,
    },
    "test": {
        "private": 0x04358394,
        "public": 0x043587CF,
    },
}

def _serialize_extended_key(key, chaincode, depth=0, parent=None, index=0, 
                            network="main"):
    """Serialize an extended private *OR* public key, as spec by bip-0032.
    :param key: The public or private key to serialize. Note that if this is
                a public key it MUST be compressed.
    :param depth: 0x00 for master nodes, 0x01 for level-1 derived keys, etc..
    :param parent: The parent pubkey used to derive the fingerprint, or the
                   fingerprint itself None if master.
    :param index: The index of the key being serialized. 0x00000000 if master.
    :param chaincode: The chain code (not the labs !!).
    :return: The serialized extended key.
    """
    for param in {key, chaincode}:
        assert isinstance(param, bytes)
    for param in {depth, index}:
        assert isinstance(param, int)
    if parent:
        assert isinstance(parent, bytes)
        if len(parent) == 33:
            fingerprint = _pubkey_to_fingerprint(parent)
        elif len(parent) == 4:
            fingerprint = parent
        else:
            raise ValueError("Bad parent, a fingerprint or a pubkey is"
                             " required")
    else:
        fingerprint = bytes(4)  # master
    # A privkey or a compressed pubkey
    assert len(key) in {32, 33}
    if network not in {"main", "test"}:
        raise ValueError("Unsupported network")
    is_privkey = len(key) == 32
    prefix = ENCODING_PREFIX[network]["private" if is_privkey else "public"]
    extended = prefix.to_bytes(4, "big")
    extended += depth.to_bytes(1, "big")
    extended += fingerprint
    extended += index.to_bytes(4, "big")
    extended += chaincode
    if is_privkey:
        extended += b'\x00'
    extended += key
    return extended

def get_master_xpriv(hd_key):
    """Get the encoded extended private key of the master private key"""
    key = hd_key.keys.secret.to_bytes(32, 'big')
    depth = 0
    parent_fingerprint = 0
    index = 0
    master_chaincode = hd_key.chaincode
    network = "main"
    extended_key = _serialize_extended_key(key, master_chaincode, depth,
                                               parent_fingerprint,
                                               index, network)
    return util.encode_base58_checksum(extended_key)

def get_master_xpub(hd_key):
    """Get the encoded extended private key of the master private key"""
    key = hd_key.keys.point.sec()
    depth = 0
    parent_fingerprint = 0
    index = 0
    master_chaincode = hd_key.chaincode
    network = "main"
    extended_key = _serialize_extended_key(key, master_chaincode, depth,
                                               parent_fingerprint,
                                               index, network)
    return util.encode_base58_checksum(extended_key)

In [ ]:
xpriv = get_master_xpriv(hd_root)
xpub = get_master_xpub(hd_root)
print(xpriv)
print(xpub)

## Dérivation d'une clé privée enfant non blindée ("unhardened")
Avec notre première clé étendue, nous pouvons commencer à dériver en ajoutant un index et en hashant la clé (privée ou publique), le chaincode et l'index.
L'index est un entier compris entre 0 et 2^32. Les valeurs entre 0 et 2^31 sont réservées aux dérivations "non blindées", ce qui signifie que si une clé privée enfant est connu, on peut retrouver la clé privée du parent, et donc aussi les clé soeurs.
![](pics/hd3.jpg)

In [ ]:
class child_key:
    def __init__(self, seed):
        if len(seed) != 64:
            raise ValueError("Provided seed should have length of 64")
        self.keys = ecc.PrivateKey(int.from_bytes(seed[:32], 'big'))
        self.chaincode = seed[32:]

    def __repr__(self):
        return f"The private key is {self.keys.wif()}\n" + \
            f"The public key is {binascii.hexlify(self.keys.point.sec())}\n" + \
            f"The chaincode is {binascii.hexlify(self.chaincode)}"


In [ ]:
def derive_unhardened_private_child(parent, index):
    if index < 0 or index >= 2 ** 31:
        raise ValueError("index must be comprised between 0 and 2 ** 31 - 1")
    privkey = parent.keys.secret
    pubkey = parent.keys.point.sec()
    payload = hmac.new(parent.chaincode, pubkey + index.to_bytes(4, "big"),
                       hashlib.sha512).digest()
    child_priv = ecc.PrivateKey((int.from_bytes(payload[:32], 'big') + privkey) % ecc.N)
    assert child_priv.point.x is not 0
    hd_child = child_priv.secret.to_bytes(32, 'big') + payload[32:]
    return child_key(hd_child)

In [ ]:
print("child1\n")
child1 = derive_unhardened_private_child(hd_root, 0)
print(child1)
print("\nchild2\n")
child2 = derive_unhardened_private_child(hd_root, 1)
print(child2)
print("\nchild3\n")
child3 = derive_unhardened_private_child(hd_root, 2)
print(child3)

## Le risque de la dérivation unhardened
![](pics/hd6.jpg)

# Dérivation d'une clé privée enfant blindée ("hardened")
Les valeurs d'index comprises entre 2^31 et 2^32 sont réservées aux dérivations "blindées". Même si une clé privée enfant est connu, il est impossible de retrouver le parent.
![](pics/hd7.jpg)

In [ ]:
def derive_hardened_private_child(parent, index):
    if index < 2 ** 31:
        raise ValueError("index must be greater or equal to 2 ** 31")
    privkey = parent.keys.secret
    pubkey = parent.keys.point.sec()
    payload = hmac.new(parent.chaincode, b'\x00' + privkey.to_bytes(32, 'big') + index.to_bytes(4, "big"),
                       hashlib.sha512).digest()
    child_priv = ecc.PrivateKey((int.from_bytes(payload[:32], 'big') + privkey) % ecc.N)
    assert child_priv.point.x is not 0
    hd_child = child_priv.secret.to_bytes(32, 'big') + payload[32:]
    return child_key(hd_child)

In [ ]:
print("child1\n")
child1 = derive_hardened_private_child(hd_root, 2 ** 31)
print(child1)
print("\nchild2\n")
child2 = derive_hardened_private_child(hd_root, (2 ** 31) + 1 )
print(child2)
print("\nchild3\n")
child3 = derive_hardened_private_child(hd_root, (2 ** 31) + 2)
print(child3)

## Dérivation sur plusieurs niveaux

In [ ]:
print("gchild1-1\n")
gchild1_1 = derive_hardened_private_child(child1, 2 ** 31)
print(gchild1_1)
print("\ngchild1-2\n")
gchild1_2 = derive_hardened_private_child(child1, (2 ** 31) + 1 )
print(gchild1_2)
print("\ngchild1-3\n")
gchild1_3 = derive_hardened_private_child(child1, (2 ** 31) + 2)
print(gchild1_3)

In [ ]:
print("gchild2-1\n")
gchild2_1 = derive_hardened_private_child(child2, 2 ** 31)
print(gchild2_1)
print("\ngchild2-2\n")
gchild2_2 = derive_hardened_private_child(child2, (2 ** 31) + 1 )
print(gchild2_2)
print("\ngchild2-3\n")
gchild2_3 = derive_hardened_private_child(child2, (2 ** 31) + 2)
print(gchild2_3)

# Dérivation d'une clé publique
L'un des avantages de la dérivation est que l'on peut dériver directement des clés publiques sans exposer la clé privée. Cela est très utilisée lorsque l'on doit générer des adresses sur un serveur non sécurisé.
![](pics/hd4.jpg)

In [ ]:
class stripped_child_key:
    def __init__(self, seed):
        self.pub = ecc.S256Point.parse(seed[:33])
        self.chaincode = seed[33:]

    def __repr__(self):
        return f"The public key is {binascii.hexlify(self.pub.sec())}\n" + \
            f"The chaincode is {binascii.hexlify(self.chaincode)}"


In [ ]:
def derive_unhardened_public_child(parent, index):
    if index < 0 or index >= 2 ** 31:
        raise ValueError("index must be comprised between 0 and 2 ** 31 - 1")
    pubkey = parent.keys.point
    payload = hmac.new(parent.chaincode, pubkey.sec() + index.to_bytes(4, "big"),
                       hashlib.sha512).digest()
    child_pub = ecc.PrivateKey(int.from_bytes(payload[:32], 'big')).point + pubkey
    assert child_pub.x is not 0
    hd_child = child_pub.sec() + payload[32:]
    return stripped_child_key(hd_child)

In [ ]:
print("child1\n")
child1 = derive_unhardened_public_child(hd_root, 0)
child_priv1 = derive_unhardened_private_child(hd_root, 0)
print(child1)
print(child1.pub == child_priv1.keys.point)
print("\nchild2\n")
child2 = derive_unhardened_public_child(hd_root, 1 )
child_priv2 = derive_unhardened_private_child(hd_root, 1)
print(child2)
print(child2.pub == child_priv2.keys.point)
print("\nchild3\n")
child3 = derive_unhardened_public_child(hd_root, 2)
child_priv3 = derive_unhardened_private_child(hd_root, 2)
print(child3)
print(child3.pub == child_priv3.keys.point)

## Sources et crédits

* [darosior pour le code de BIP32](https://github.com/darosior/python-bip32)
* [evoskuil pour les images](https://github.com/libbitcoin/libbitcoin-system/wiki/Addresses-and-HD-Wallets)
* [Trezor pour le code des mnemonics en Python](https://github.com/trezor/python-mnemonic/tree/master/mnemonic)